# Machine learning: regression

We'll try to predict missing well logs using regression.

The data are from Colorado. We've already loaded the data into a CSV.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

In [ ]:
well_logs = '../data/Colorado_well_data.csv'

In [ ]:
df = pd.read_csv(well_logs, index_col=0)

In [ ]:
df.describe()

Look at the counts: some of the features have some NaNs. It looks like we won't lose too much data by doing a `dropna`...

In [ ]:
df = df.dropna()

In [ ]:
df.describe()

In [ ]:
df.head()

I'd prefer it if the well names were integers not floats.

In [ ]:
df.Well = df.Well.astype(int)

In [ ]:
df.head()

# Visual inspection of the data space

In [ ]:
well = 10

features = ['CAL', 'SP', 'GR', 'RES', 'NPHI', 'RHOB']
target = 'DT'

fig, axs = plt.subplots(ncols=len(features)+1, sharey=True, figsize=(8,8))

for ax, feature in zip(axs, features):
    ax.plot(df.loc[df.Well==well, feature], df.loc[df.Well==well, 'Depth'])
    ax.set_title(feature)
axs[-1].plot(df.loc[df.Well==well, target], df.loc[df.Well==well, 'Depth'], color='red')
axs[-1].set_title(target)
axs[-1].invert_yaxis()

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
Make a 'log<sub>10</sub> resisitivity' to deal with the usual RES distribution. Call it `LogRes`.
</div>

In [ ]:
df.describe()

In [ ]:
sns.distplot(df.LogRes)

And update the `features` list:

In [ ]:
features.remove('RES')
features.append('LogRes')

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
Now look at the gamma ray. Are the very high values coming from one well or from lots of places? 

Decide how to fix the gamma ray. For example, you could:

- Remove one or more wells with bad GR.
- Remove only the rows with very high values.
- Clip the GR, e.g. using `pd.Series.clip()`.
</div>

In [ ]:
sns.distplot(df.GR)

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
What is causing the long tail in the NPHI data? Is it spikes or a bad log?

Decide how best to fix the NPHI, limiting its range to the interval 0 to 0.5.
</div>

In [ ]:
sns.distplot(df.NPHI)

The distributions should now look something like this:

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

In [ ]:
df.describe()

## Method chaining

Some people like chaining Pandas' methods into long 'pipelines'. 

In [ ]:
df.query('NPHI > 0.4')  # df.query() is nice... and it returns a df

In [ ]:
def log_res(df):
    df['LogRes'] = np.log10(df.RES)
    return df

dz = (pd.read_csv(well_logs, index_col=0)
        .dropna()
        .pipe(log_res)
        .query('(Well != 3) & (Well != 5)')
     )

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>Exercise</h3>

This implemented some of our pipeline; can you implement the other things we did?

- Clip the GR curve, eg at 400 API.
- Fix the NPHI by applying the `fix_nphi` function we already made.
- Apply limits to the NPHI curve as before (0 to 0.5).
</div>

In [ ]:
def fix_gamma(df):
    
    # YOUR CODE HERE

    return df


dz = (pd.read_csv(well_logs, index_col=0)
        .dropna()
        .pipe(log_res)
        .query('(Well != 3) & (Well != 5)')

        # YOUR CODE HERE

     )

In [ ]:
# You should end up with 372611 records.
len(dz) == 372611

## Feature engineering

In [ ]:
features = ['GR', 'NPHI', 'RHOB', 'LogRes']
target = 'DT'

### Combination features

In [ ]:
from itertools import combinations

combs = combinations(features[:4], 2)
list(combs)

In [ ]:
combs = combinations(features[:4], 2)
for f1, f2 in combs:
    new_feature = f'{f1}.{f2}'
    df[new_feature] = df[f1] * df[f2]
    if new_feature not in features:
        features.append(new_feature)

In [ ]:
features

### Empirical features

In [ ]:
df['InvGardner'] = 108 * df.RHOB**4

if 'InvGardner' not in features:
    features.append('InvGardner')

### Smoothed features

In [ ]:
df['NPHI_smooth'] = 0
for name, group in df.groupby('Well'):
    df.NPHI_Smooth.loc[group.index] = np.convolve(group.NPHI.copy(),
                                                  np.ones(21)/21,
                                                  mode='same')

if 'NPHI_smooth' not in features:
    features.append('NPHI_smooth')

In [ ]:
plt.figure(figsize=(15,3))
df.NPHI[:1000].plot()
df.NPHI_Smooth[:1000].plot()

### Nonlinear transformations

In [ ]:
df['NPHI_sq'] = df.NPHI ** 2.0

if 'NPHI_sq' not in features:
    features.append('NPHI_sq')

In [ ]:
# df['RHOB_sq'] = df.RHOB ** 2.0
# df['RHOB_sr'] = df.RHOB ** 0.5
# Etc.

In [ ]:
features

# Split the dataset

In [ ]:
df.Well.unique()

How many wells is that?

In [ ]:
len(df.Well.unique())

Let's start by training on the first six wells only.

In [ ]:
n = 8  # We'll come back and change this number.

In [ ]:
df_train = df[df.Well <= n].copy()
df_val = df[(df.Well >= 70) & (df.Well < 85)].copy()   # 12 wells
df_test = df[df.Well >= 85].copy()  # 10 wells

## Check the distributions

We'd like to make sure the distributions of the 3 datasets are comparable.

In [ ]:
features

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(15,3))

for ax, feature in zip(axs, features):
    sns.distplot(df_train[feature], ax=ax)
    sns.distplot(df_val[feature], ax=ax)
    sns.distplot(df_test[feature], ax=ax)
    ax.set_yticklabels([])

## Make `X` and `y`

In [ ]:
X_train = df_train[features].values
y_train = df_train[target].values

X_val = df_val[features].values
y_val = df_val[target].values

X_test = df_test[features].values
y_test = df_test[target].values

## Standardize

It's sensible to standardize the data before linear regression. This transforms all of the features to their Z-scores. That is, we subtract the mean and divide by the standard deviation. 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Train a model

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

regr = LinearRegression()

regr.fit(X_train, y_train)

In [ ]:
df_val['DT_pred_LR'] = regr.predict(X_val)

df_val.head()

In [ ]:
def plot_track(df, idx, *cols):
    fig, ax = plt.subplots(1,1)
    fig.set_size_inches(15,3)
    depths = df.loc[df.Well == idx, 'Depth']
    for col in cols:
        ax.plot(depths, df.loc[df.Well == idx, col], lw=1.5, label=col)
    ax.set_xlim(1300, 2400)
    ax.set_ylim(40, 140)
    plt.legend()
    return

In [ ]:
@interact(idx=(df_val.Well.unique().min(), df_val.Well.unique().max(), 1))
def plot_different_wells(idx=76):
    plot_track(df_val, idx, 'DT', 'DT_pred_LR')
    return

# Evaluation metrics

Two convenient ways to evaluate regressions are with the $R^2$ score...

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

r2_score(df_val.DT, df_val.DT_pred_LR)

And the RMS error:

In [ ]:
np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_LR))

## Other linear regressors

Linear regression by ordinary least squares (`LinearRegression` model) tries to find the best weights or coefficients, and their values are unconstrained. 

Sometimes we want to constrain them a little, to try to make a 'simpler' model and prevent overfitting.

For example, `Ridge` adds an L2 penalty to the weights. In other words, it tries to keep the weight vector as small ('short', or low magnitude) as possible. In geophysics, this is sometimes referred to as [Tikhonov regularization](https://en.wikipedia.org/wiki/Tikhonov_regularization).

In [ ]:
lr_coef = regr.coef_

In [ ]:
regr = Ridge(alpha=1.0)

regr.fit(X_train, y_train)

df_val['DT_pred_L2'] = regr.predict(X_val)

print('r2_score ', r2_score(df_val.DT, df_val.DT_pred_L2))
print('RMS error', np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_L2)))

In [ ]:
l2_coef = regr.coef_

Alternatively, we can try to minimize the L1 norm of the weight vector. This tries to keep as many coefficients as possible at 0. If you believe there to be only a few important features, it would be sensible to try this.

This regressor is called [LASSO](https://en.wikipedia.org/wiki/Lasso_(statistics)).

In [ ]:
regr = Lasso(alpha=1.0)

regr.fit(X_train, y_train)

df_val['DT_pred_L1'] = regr.predict(X_val)

print('r2_score ', r2_score(df_val.DT, df_val.DT_pred_L1))
print('RMS error', np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_L1)))

In [ ]:
l1_coef = regr.coef_  

In [ ]:
for coef, data in {'lr': lr_coef, 'l2': l2_coef, 'l1': l1_coef}.items():
    plt.plot(data, lw=2, label=coef)
plt.axhline(0, c='k', lw=1)
plt.legend()

## Check error distribution

In particular, we want to check that:

1. The errors are normally distributed with a zero mean.
1. The variance of the errors is not correlated with the parameters.

There's some good advice about normality tests in [this article by Jason Brownlee](https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/).

First we'll just use visual inspection:

In [ ]:
residuals = df_val['DT_pred_LR'] - df_val['DT']

In [ ]:
sns.distplot(residuals)
plt.axvline(0, color='k', lw=0.5)
plt.axvline(residuals.mean(), color='r')
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: QQ plot

A quantile-quantile plot generates an idealized distribution, in this case a Gaussian. The idealized samples are divided into quantiles, then each data point in the sample is paired with a similar member from the idealized distribution. The line `'s'` represents the standard 'normal' distribution.

In [ ]:
from statsmodels.graphics.gofplots import qqplot

qqplot(residuals, line='s')
plt.axvline(0, color='k', lw=0.5)
plt.axhline(0, color='k', lw=0.5)
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: Shapiro&ndash;Wilk test

Not convinced about this &mdash; seems like most large samples don't fit. `p` just gets very small.

In [ ]:
from scipy.stats import shapiro

res_shuf = residuals.values
np.random.shuffle(res_shuf)

stat, p = shapiro(res_shuf[:500])
print(f'Statistics = {stat:.3f}, p = {p:.3f}')

alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

#### Homoscedasticity: visual inspection

We want to check that the variance of the errors is not correlated with our parameters.

If they are correlated (if the plots below show points with narrow spread at one end and wide at the other), then there are nonlinearities in the data that are not captured by the model. It could be that outliers are skewing the distribution.

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(16,4), sharey=True)

for ax, feature in zip(axs, features):
    ax.scatter(df_val[feature], residuals, s=1, alpha=0.1)
    ax.set_xlabel(feature)
    ax.axhline(0, color='k', lw=0.5)
    ax.grid(color='k', alpha=0.15)

## Coefficients

**If the features have been standardized**, then we can interpret the learned coefficients (or parameters, or weights if you prefer) as feature importance.

In [ ]:
np.set_printoptions(suppress=True)
regr.coef_

In [ ]:
regr.intercept_

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
Can you make a list of the features ordered by their coefficients?
</div>

## `statsmodels`: Confidence interval and prediction interval

Often, we'd like to know the range of possible responses. Scikit-learn is for predictions, and doesn't have a lot of tools for in-sample statistics. For that, we need to use `statsmodels` or some other statistical package. 

Check out [this notebook by Matteo Niccoli](https://github.com/mycarta/Data-science-tools-petroleum-exploration-and-production/blob/master/Python/notebooks/Python_OLS_confidence_interval_and_prediction_interval.ipynb)

For regression of a variable y on a single independent variable x, you can also use `seaborn`, e.g. [see this help page](https://seaborn.pydata.org/tutorial/regression.html).

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table

X_train_s = sm.add_constant(X_train)  # Intercept not included by default.

reg = sm.OLS(y_train, X_train_s).fit()  # Fit the model.

_, data, columns = summary_table(reg, alpha=0.05)  # Get the results, including CI, PI.

ds = pd.DataFrame(data, columns=map(lambda s: s.replace('\n', ' '), columns))

ds.head()

**Confidence interval** is the range we expect the **mean** to be within, for a given confidence level (default is `alpha = 0.5` or a confidence level of 95%). 

**Prediction interval** is the range we expect **a given observation** to be within at that confidence level.

Let's also check the predictive power of this model.

In [ ]:
X_val_s = sm.add_constant(X_val)

df_val['DT_pred_SM'] = reg.predict(X_val_s)

These should be more or less exactly like the OLS results from `sklearn`...

In [ ]:
r2_score(df_val.DT, df_val.DT_pred_SM)

In [ ]:
np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_SM))

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>Exercise: Try using a deep neural network</h3>

Have a go at doing this regression using `sklearn`'s neural network, or using TensorFlow if you prefer. See if you can beat the linear regression.
</div>

In [ ]:
from sklearn.neural_network import MLPRegressor

# YOUR CODE HERE

In [ ]:
@interact(idx=(df_val.Well.unique().min(), df_val.Well.unique().max(), 1))
def plot_different_wells(idx=76):
    plot_track(df_val, idx, 'DT', 'DT_pred_LR', 'DT_pred_NN', 'DT_pred_TF')
    return